# **Extracción de tweets con Twint** #

In [5]:
import pandas as pd
#import twint
#import nest_asyncio
#nest_asyncio.apply()
import os
import shutil
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords

In [16]:
def Crear_carpeta(Rutabase, name_carpeta):
    # Se define el nombre de la carpeta o directorio a crear
    directorio = Rutabase + name_carpeta
    if os.path.exists(directorio):
        shutil.rmtree(directorio)
        os.makedirs(directorio)
        print("Se reemplazo el directorio %s " % directorio)
    else:
        try:
            os.mkdir(directorio)
        except OSError:
            print("La creación del directorio %s falló" % directorio)
        else:
            print("Se ha creado el directorio: %s " % directorio)

In [17]:
def Buscar_Palabra(cadena, palabra_incluida):
    lista_cadenas = [cadena]
    #print(cadena.find(palabra_incluida))
    if cadena.find(palabra_incluida) < 0:
        lista_cadenas.append(palabra_incluida)
    return lista_cadenas
    

## Configuracion de Twint ##

*Se establece el codigo de configuracion para recuperacion los tweets mediante usernames:*

In [18]:
def search_tweets(i, since, until):
    c = twint.Config()
    #c.Search = tag
    c.Username = i
    c.Language = 'es'
    c.Since = since
    c.Until = until
    c.Limit = 100
    c.Pandas = True
    c.Hide_output= True
    twint.run.Search(c)
    df_twint = twint.storage.panda.Tweets_df
    return df_twint

#search_tweets('MartinVizcarraC', '2015-05-01', '2015-06-01')

*Se establece el codigo de configuracion para recuperacion los tweets mediante hashtags:*

In [19]:
def search_tweets_hash(i, since, until):
    tag = Buscar_Palabra(i,'Peru')
    c = twint.Config()
    c.Search = tag
    c.Language = 'es'
    c.Since = since
    c.Until = until
    c.Limit = 100
    c.Pandas = True
    c.Hide_output= True
    twint.run.Search(c)
    df_twint = twint.storage.panda.Tweets_df
    return df_twint

#search_tweets('MartinVizcarraC', '2015-05-01', '2015-06-01')

*Mediante el metodo **afinar_busqueda**, se configura la busqueda para que devuelva la cantidad de tweets mayores a 0.*

In [37]:
def afinar_busqueda(i,since,until, parametro):
    df_twint = pd.DataFrame()
    k = 0
    while len(df_twint) < 1:
        #print(k)
        #df_twint = search_tweets(i, since, until)
        df_twint = search_tweets_hash(i, since, until)
        if len(df_twint) > 1 or k == 2:
            break
        k+=1
    return df_twint
           
#afinar_busqueda('MartinVizcarraC', '2021-02-01', '2021-03-01',5)

## Seleccionar usernames ##
*Se define las cuentas verificadas de Twiter para recopilar sus opiniones, en este caso utilizar el user_id. En el caso del escenario politico, los personajes seleccionados fueron los candidatos politicos de las elecciones presidenciales 2021 del Peru. Mientras que en el escenario de la pandemia, las cuentas seleccionadas son las principales entidades de salud en el Peru; asi como personajes relevantes (Ministros de Salud, medicos, etc).*

In [21]:
user_politica=['PedroCastilloTe','KeikoFujimori','MartinVizcarraC','Vero_Mendoza_F','DanielUrresti1','FSagasti',
               'rlopezaliaga1','yonhy_lescano','CesarAcunaP','HDeSotoPeru','George_Forsyth','MerinoDeLama',
               'ONPE_oficial', 'JNE_Peru']
user_pandemia=['Minsa_Peru', 'ougarteu', 'victorzamora','EsSaludPeru', 'drhuerta', 'SuSaludPeru', 'SISPeruOficial',
               'HCevallosFlores', 'ErnesBustamante','maguina_ciro','A_Aguinaga','CMP_PERU', 'CayetanoHeredia',
               'EdMalagaTrillo']

## Seleccionar hashtags ##
*Se definen diferentes hashtags que fueron utilizados durante el desarrollo de acontecimientos politicos y de pandemia.*

In [33]:
hash_politico = ['#EleccionesPeru2021','#FranciscoSagasti','#VioletaBermudez','#BicentenarioPeru','#AnaJara',
                 '#WaldoMendoza', '#MirthaVásquez', '#felizbicentenarioPeru', '#sagasti', '#FelizDiaPeru', '#FelicesFiestasPatrias']

In [6]:
hash_pandemia = ['#Covid-19','#pandemia','#coronavirus','#vacunas','#VacúnateYa', '#COVID19', '#PongoElHombro', 
                 '#LaVacunaEsVida', '#vacuna', '#vacunaton', '#NoBajemosLaGuardia',
       '#VamosaSalirAdelante', '#vacunacion', '#sialavacuna',
       '#VacunaFest', '#TodasLasVacunasSirven', '#VacunaCOVID19',
       '#antivacunas', '#NuevaCepa', '#Sinopharm', '#YoApoyoaBeto',
       '#ErnestoBustamante', '#EsSaludPeru', '#pandemia', '#covid',
       '#covid_19', '#SagastiGenocida', '#SuizaLab', '#AstraZeneca',
       '#Vacunate', '#Vacunagate', '#Minsa', '#Pfizer', '#delta',
       '#variante', '#PfizerBiontech', '#Moderna', '#VacunacionPeru',
       '#VacunateYa', '#Minsa_Peru', '#Omicron', '#yomequedoencasa',
       '#SARSCoV2', '#confinamiento', '#cuarentena', '#SaludEnTuVIda', '#mascarillas','#MedidasPreventivas',
       '#LavadoDeManos', '#UsoDeCubrebocas', '#SanaDistancia',
       '#PrevenirEsSalud','#CuidarteEsCuidarnos', '#SaludMental','#PCR']


In [7]:
hash_pandemia1 = ['#Covid-19','#pandemia','#coronavirus','#VacúnateYa', '#COVID19', '#PongoElHombro', 
                 '#LaVacunaEsVida', '#vacunaton', '#NoBajemosLaGuardia',
       '#VamosaSalirAdelante', '#vacunacion', '#sialavacuna',
       '#VacunaFest', '#TodasLasVacunasSirven', '#VacunaCOVID19',
       '#antivacunas', '#NuevaCepa', '#Sinopharm', '#EsSaludPeru', '#covid', '#SuizaLab', '#AstraZeneca',
       '#Vacunate', '#Vacunagate', '#Minsa', '#Pfizer', '#delta','#Moderna', '#VacunacionPeru',
       '#VacunateYa', '#Minsa_Peru', '#Omicron', '#yomequedoencasa',
       '#SARSCoV2', '#confinamiento', '#cuarentena', '#SaludEnTuVIda', '#mascarillas',
        '#PongoElHombro', '#VacunaCOVID19', "#VacunasPorLaVida" , "#VacunarseEsNecesario",
       '#LavadoDeManos', '#UsoDeCubrebocas', '#SanaDistancia',
       '#PrevenirEsSalud','#CuidarteEsCuidarnos', '#SaludMental','#PCR']

In [8]:
hash_pandemia_2 = ['#PongoElHombro', '#VacunaCOVID19', "#VacunasPorLaVida" , "#VacunarseEsNecesario", '#LavadoDeManos', '#UsoDeCubrebocas', '#SanaDistancia',
       '#PrevenirEsSalud','#CuidarteEsCuidarnos', '#SaludMental','#PCR']

In [9]:
print(len(hash_pandemia), len(hash_pandemia1), len(hash_pandemia_2))

55 49 11


## Recuperacion de Tweets ##
*Los tweets que son recuperandos son almacenados en un dataframe pos separado y posteriormente estos son unificados en dataframe general para hacer la limpieza de datos.*

In [23]:
def Generar_Tweets(lista, Rutabase, year_1, year_2 ):
    lista = pd.unique(lista)
    numero_tweets = 0
    for i in lista:
        sub_df = pd.DataFrame()
        for j in range(year_1,year_2):
            df = pd.DataFrame()
            for x in range(1,13):
                print(x)
                if x < 12:
                    since = str(j) + '-'+ str(x) +'-01'
                    until = str(j) + '-'+ str(x + 1) +'-01'
                else:
                    #print('aqui')
                    since = str(j) + '-'+ str(x) +'-01'
                    until = str(j + 1) + '-'+ str(1) +'-01'
                df = afinar_busqueda(i, since, until, 5)
                #print(since, until)
                    #print(len(df))
                sub_df = pd.concat([sub_df, df],ignore_index=False)
                Rutarel =  i + '.csv'
        Rutasol = os.path.join(Rutabase, Rutarel)
        sub_df.to_csv(Rutasol, index = False)
        print('Se ha creado correctamente el ' + i + '.csv, recuperando ' + str(len(sub_df)) + ' tweets')
        sub_df.to_csv(i + '.csv')
#Generar_Tweets(user_pandemia,2021,2022)

In [65]:
#file_size = os.path.getsize(r'C:\test\file1.txt') 
def SeleccionarTweets(lista, Rutabase):
    total_df = pd.DataFrame() 
    count = 0
    for i in lista:
        Rutarel =  i + '.csv'
        Rutasol = os.path.join(Rutabase, Rutarel)
        if (os.path.getsize(Rutasol) > 2):
            print( Rutasol, os.path.getsize(Rutasol))
            df = pd.read_csv(Rutasol, delimiter=',')
            print(len(df))
            new_df = df[df['language'] == 'es']
            print('Se ha recuperado ' + str(len(new_df)) + ' tweets en el idioma espanhol')
            new_df = new_df.reset_index(drop = True)
            #new_df = Eliminar_Repetidos(new_df, 'tweet', 'spanish', 0.5)
            #print('Se han eliminado los repetidos del archivo '+ i +'.csv'+ ' , el nuevo numero de tweets es ' + str(len(new_df)))
            #count = count + len(df)
            total_df = pd.concat([total_df, new_df],ignore_index=False)
    #print(total_df.shape)
    total_df = total_df.reset_index(drop = True)
    print('Se ha recuperado ' + str(len(total_df)) + ' tweets en total' )
    return total_df
   
#def Eliminar_Reptidos_Archivo(df):
    
#df[(df['date'] > '2021-04-01') & (df['date'] < '2021-10-01')]

In [25]:
def Recuperar_Indice(lista_indices):
    index = []
    for i in lista_indices:
        index.append(i[1])
    index = pd.unique(index)
    index = list(index)
    return index

## Eliminar duplicados##
*Se establece la **similidaridad de coseno** para identificar el grado de similitud entre dos tweets. Este metodo sirve para eliminar los tweets duplicados.*

In [26]:
def Eliminar_Repetidos(df, label, language, threshold):
    tfidf = TfidfVectorizer(stop_words = stopwords.words(language))
    # Construct the TF-IDF matrix
    tfidf_matrix = tfidf.fit_transform(df[label])
    #print(shape(tfidf_matrix))
    # Generate the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    #print(cosine_sim)
    #Lista que recupera los tweets
    A = []
    for i in range(0,len(cosine_sim)):
        for k in range(0,len(cosine_sim)):
            if ((k>i) and (cosine_sim[i][k] >= threshold)):
                #print(i,k)
                A.append([i,k])
                
    indices_tweets = Recuperar_Indice(A)
    df = df.drop(indices_tweets)
    #print(df)
    df = df.reset_index(drop = True)
    print('Se han recuperado ' + str(len(df)) + ' tweets')
    return df
    

In [46]:
#Rutabase = r'C:/Users/USER/Documents/Git/SATwint/'
Rutabase =  r'C:/Users/claud/OneDrive/Documentos/GITHUB/TESIS/Extraction_with_Twint/'
#nam3e_carpeta = 'pandemia/'
def Obtener_dataset(lista_tweet, name_carpeta, label, language, name, year_1, year_2):
    #Crear_carpeta(Rutabase, name_carpeta)
    Ruta = Rutabase + name_carpeta
    #Generar_Tweets(lista_tweet, Ruta, year_1, year_2)
    df = SeleccionarTweets(lista_tweet, Ruta)
    df.to_csv(name + '.csv') 
    df = Eliminar_Repetidos(df, label, language, 0.5)
    df.to_csv(name + 'Limpio.csv')  
    return df

In [70]:
final_df = Obtener_dataset(hash_pandemia,'Pandemia_hash/', 'tweet', 'spanish', 'df_pandemia_hashv2', 2020,2022)
#final_df = Obtener_dataset(user_pand,'pandemia/', 'tweet', 'spanish', 'df_elecciones' )
#final_df.to_csv('new_pand.csv')

C:/Users/claud/OneDrive/Documentos/GITHUB/TESIS/Extraction_with_Twint/Pandemia_hash/#Covid-19.csv 255692
419
Se ha recuperado 329 tweets en el idioma espanhol
C:/Users/claud/OneDrive/Documentos/GITHUB/TESIS/Extraction_with_Twint/Pandemia_hash/#pandemia.csv 248064
352
Se ha recuperado 329 tweets en el idioma espanhol
C:/Users/claud/OneDrive/Documentos/GITHUB/TESIS/Extraction_with_Twint/Pandemia_hash/#coronavirus.csv 208913
314
Se ha recuperado 250 tweets en el idioma espanhol
C:/Users/claud/OneDrive/Documentos/GITHUB/TESIS/Extraction_with_Twint/Pandemia_hash/#COVID19.csv 302565
451
Se ha recuperado 342 tweets en el idioma espanhol
C:/Users/claud/OneDrive/Documentos/GITHUB/TESIS/Extraction_with_Twint/Pandemia_hash/#NoBajemosLaGuardia.csv 50704
78
Se ha recuperado 74 tweets en el idioma espanhol
C:/Users/claud/OneDrive/Documentos/GITHUB/TESIS/Extraction_with_Twint/Pandemia_hash/#VamosaSalirAdelante.csv 8525
13
Se ha recuperado 13 tweets en el idioma espanhol
C:/Users/claud/OneDrive/Docume

In [21]:
final_df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [25]:
final_df['hashtags'].unique()

array(["['coronavirus']", "['coronavirus', 'tusaludprimero']",
       "['coronavirus', 'minsa']", ..., "['buenanoticia', 'chorrillos']",
       "['samu']",
       "['trastornodellenguaje', 'saludmental', 'derechodetodos']"],
      dtype=object)

In [27]:
final_df[final_df['hashtags'] == 'coronavirus']

,id,conversation_id,created_at,date,timezone,place,tweet,language,hashtags,cashtags,...,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
